<a href="https://colab.research.google.com/github/sujithgunturu/Math799/blob/master/helenaodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import pandas as pd
df = pd.read_csv('/content/Weather_data_2020.csv')
x = np.linspace(0,100, 100)

# RELHUM2MAVG
y = df["RELHUM2MAVG"][:100]
Wht = interpolate.interp1d(x, y, fill_value="extrapolate")


#WSPD2MAVG
y = df["WSPD2MAVG"][:100]
Wwt = interpolate.interp1d(x, y, fill_value="extrapolate")


#PRECIP
y = df["PRECIP"][100:200]
Wrt = interpolate.interp1d(x, y, fill_value="extrapolate")


# (TEMP2MAVG)
y = df["TEMP2MAVG"][:100]
Wat = interpolate.interp1d(x, y, fill_value="extrapolate")


#SR
y = df["SR"][:100]
Wst = interpolate.interp1d(x, y, fill_value="extrapolate")


#PRESSUREAVG
y = df["PHOTOPERIOD"][:100]
Wpt1 = interpolate.interp1d(x, y, fill_value="extrapolate")

#PHOTOPERIOD
y = df["PRESSUREAVG"][:100]
Wpt2 = interpolate.interp1d(x, y, fill_value="extrapolate")

#TEMP2MMIN
y = df["TEMP2MMIN"][:100]
Wnt = interpolate.interp1d(x, y, fill_value="extrapolate")


#TEMP2MMAX
y = df["TEMP2MMAX"][:100]
Wxt = interpolate.interp1d(x, y, fill_value="extrapolate")

def rhs(SV, t, 
        theta_vr,theta_vd, theta_vo,
        theta_vs, theta_vq, theta_ps, 
        theta_pr, theta_pm, theta_tr, 
        theta_td, theta_t0,theta_alpha, 
        theta_el, theta_k, theta_sr, 
        theta_sm,theta_gx, theta_m, 
        theta_gb,theta_ts, theta_rue,
        theta_a, Sxl):
        #k_m, k_i,k_e ):
  Svt, Sat, Slt, Sbt, Sgt, Syt, Swt = SV

  
  dsvt_dt = (1 + np.exp(-1 * theta_vr * theta_vd))/(np.exp(-1 * theta_vr * (Wat(t) - theta_vo + theta_vd) + np.exp(theta_vr) * (Wat(t) - theta_vo) - 1)+1 )
  f_vt = 1-theta_vs*(1- 1/(1+np.exp(-10/theta_vq*(Svt- theta_vq/2))))
  f_pt = 1 - (1 - theta_ps * (1- (1 / (np.exp(-100 * (( Sat/theta_a)-1) ))) )) * (1/(1 + np.exp(-theta_pr*(Wpt1(t) - theta_pm))))
  d_tt = (1 + np.exp(-1 * theta_tr * theta_td)) / (1 + np.exp(-1 * theta_tr *(Wat(t) - theta_t0 + theta_td) + np.exp((theta_tr * (Wat(t) - theta_t0)) -1)))
  dSat_dt = f_vt * f_pt * d_tt
  f_wt = 1 / (1 + np.exp((-np.log (99)/0.175)*(Swt - 0.175)))
  dSlt_dt = Slt * (d_tt*theta_alpha*f_wt*(1 - (1/(1+np.exp(-1* 100 * ((Sat/theta_el) -1)))))* (1/np.exp(-1 * 100 *(np.exp(-1 * theta_k * Slt)))) - (1/1+np.exp(-1 * theta_sr * (Wat(t)/theta_sm - 1) )))
  dSbt_dt = theta_rue * Wst(t) * (1- np.exp(-1*theta_k * Slt) * f_wt* d_tt * (1- (1/ (1+np.exp(-100 * (Sat/theta_m -1)*(1-(1/1+np.exp(-100*((1000 * Syt) /(theta_gx *Sgt) -1)))) )) ) ))
  dSgt_dt = dSbt_dt * theta_gb * (1 / (1 + np.exp(-1 * 100 * (Sat/theta_a)-1))) * (1 - (1/(1+np.exp(-1 * 100 * (Sat/theta_ts -1)))))
  dSyt_dt = dSbt_dt * (1/ (1+np.exp(-1 * 100* (Sat/theta_a -1))))
  D_t = (4098 * 0.6108 * np.exp((17.27 * Wat(t))/Wat(t)+237.3))/(Wat(t) + 237.3)**2
  
  e_st = 0.6108 * (np.exp((17.27 * Wxt(t))/(Wxt(t) + 237.3)) + np.exp((17.27 * Wnt(t))/Wnt(t) +237.3)) /2 
  e_at = e_st * Wht(t) * 0.01
  gamma_t = 0.665 * 10**-3 * Wpt2(t)
  e0_t = ((0.408 * D_t * (Wst(t))) + (gamma_t * (900/(Wat(t) + 273) * Wwt(t)* (e_st - e_at))))/(D_t + gamma_t * (1+ 0.34 * Wwt(t)))  
  #k_ct = k_i + (1-np.exp(-1 * theta_k * Slt)) * (k_m - k_i + (k_i-k_e)/(np.exp(-1 * 100 * (Sat/theta_a - 1) ) +1) ) - ( (k_i - k_e) / np.exp(-1 * 100 * (Sat/theta_a -1) +1))
  k_ct = 0.45 + (1-np.exp(-1 * theta_k * Slt)) * (1.3 - 0.45 + (0.45-0.2)/(np.exp(-1 * 100 * (Sat/theta_a - 1) ) +1) ) - ( (0.45 - 0.2) / np.exp(-1 * 100 * (Sat/theta_a -1) +1))
  e_ct = e0_t * k_ct
  dSwt_dt = ((Wrt(t))/np.exp(-1 * 100 * (Swt/Sxl + 1) ) + 1) - (e_ct * f_wt)
  return np.array([dsvt_dt,dSat_dt,dSlt_dt,dSbt_dt,dSgt_dt,dSyt_dt, dSwt_dt])

In [ ]:
theta_vr = -0.28
theta_vd = 5
theta_vo = 4
theta_vs = 1
theta_vq = 42
theta_ps = 0.8
theta_pr = 0.44
theta_pm = 10.67
theta_tr = 0.14
theta_td = -7.6
theta_t0 = 24
theta_alpha = 0.30
theta_el = 18
theta_k = 0.75
theta_sr = 10
theta_sm =35
theta_gx = 45
theta_m = 60
theta_gb = 20
theta_ts = 15
theta_rue = 1.15
theta_a = 36
Sxl = 150
# wrong, added for time being
k_i = 0.45
k_m = 1.3
k_e = 0.2
params = (theta_vr,theta_vd, theta_vo,
        theta_vs, theta_vq, theta_ps, 
        theta_pr, theta_pm, theta_tr, 
        theta_td, theta_t0,theta_alpha, 
        theta_el, theta_k, theta_sr, 
        theta_sm,theta_gx, theta_m, 
        theta_gb,theta_ts, theta_rue,
        theta_a, Sxl)
        #k_m, k_i,k_e,
StateVars=np.array([0, 0, 0.01, 0.5, 0, 0.5, 75])
T= np.linspace(0, 100, 100)
sol = odeint(rhs,StateVars,T,args=params)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [ ]:
params = np.array(list(params))
paramserr  = params * 1.1

In [ ]:
import itertools
combinations = []
count = 0
for xs in itertools.product([params[0],paramserr[0] ], [params[1],paramserr[1]], [params[2],paramserr[2]], [params[3],paramserr[3] ], [params[4], paramserr[4]],
                            [params[5],paramserr[5] ], [params[6],paramserr[6] ], [params[7],paramserr[7] ], [params[8],paramserr[8] ], [params[9], paramserr[9]],
                            [params[10],paramserr[10] ], [params[11],paramserr[11] ], [params[12],paramserr[12] ], [params[13],paramserr[13]], [params[14], paramserr[14]],
                            [params[15],paramserr[15] ], [params[16],paramserr[16] ], [params[17],paramserr[17] ], [params[18],paramserr[18] ], [params[19], paramserr[19]],
                            [params[20],paramserr[20] ], [params[21],paramserr[21] ], [params[22],paramserr[22]]):
  combinations.append(xs)
  count += 1
  if count == 1000:
    break

In [ ]:
required_statevariables_0 = list()
required_statevariables_1 = list()
from tqdm import tqdm
for combination in tqdm(combinations):
  T= np.linspace(0, 100, 100)
  StateVars=np.array([0, 0, 0.01, 0.5, 0, 0.5, 75])
  sol = odeint(rhs,StateVars,T,args=tuple(combination))
  required_statevariables_0.append(sol[50])
  required_statevariables_1.append(sol[-1])

In [ ]:
np.save("statevariableat0",  required_statevariables_0)
np.save("statevariableat_1",  required_statevariables_1)

In [ ]:
from sympy import *
init_printing(use_unicode=True)
from sympy.matrices import Matrix
S_xl,S_vt,S_at,S_wt, theta_vr,theta_vd, theta_vo, theta_vs, theta_vq, theta_ps, theta_a, theta_pr,theta_pm, theta_tr, theta_td, theta_t0, theta_alpha, theta_el, theta_k, theta_sr,theta_sm, theta_rue, theta_m, theta_gx, theta_gb, theta_ts, W_at, W_pt1,W_pt2, S_lt, W_st, S_yt, S_gt, W_xt, W_nt, W_ht, W_wt, k_i, k_m, k_e, W_rt, S_bt = symbols("S_xl,S_vt,S_at,S_wt, theta_vr,theta_vd, theta_vo, theta_vs, theta_vq, theta_ps, theta_a, theta_pr,theta_pm, theta_tr, theta_td, theta_t0, theta_alpha, theta_el, theta_k, theta_sr,theta_sm, theta_rue, theta_m, theta_gx, theta_gb, theta_ts, W_at, W_pt1,W_pt2, S_lt, W_st, S_yt, S_gt, W_xt, W_nt, W_ht, W_wt, k_i, k_m, k_e, W_rt, S_bt")



In [ ]:
dSvt_dt = (1 + exp(-1 * theta_vr * theta_vd))/(exp(-1 * theta_vr * (W_at - theta_vo + theta_vd) + exp(theta_vr) * (W_at - theta_vo) - 1)+1 )
f_vt = 1-theta_vs*(1- 1/(1+exp(-10/theta_vq*(S_vt- theta_vq/2))))
f_pt = 1 - (1 - theta_ps * (1- (1 / (exp(-100 * (( S_at/theta_a)-1) ))) )) * (1/(1 + exp(-theta_pr*(W_pt1 - theta_pm))))
d_tt = (1 + exp(-1 * theta_tr * theta_td)) / (1 + exp(-1 * theta_tr *(W_at - theta_t0 + theta_td) + exp((theta_tr * (W_at - theta_t0)) -1)))
dSat_dt = f_vt * f_pt * d_tt
f_wt = 1 / (1 + exp((-ln(99)/0.175)*(S_wt - 0.175)))
dSlt_dt = S_lt * (d_tt*theta_alpha*f_wt*(1 - (1/(1+exp(-1* 100 * ((S_at/theta_el) -1)))))* 1/(exp(-100 *(exp(-theta_k * S_lt)-1/20))+1) - (1/(1+exp(-1 * theta_sr * (W_at/theta_sm - 1) ))))
dSbt_dt = theta_rue * W_st * (1-exp(-1*theta_k * S_lt)) * f_wt* d_tt * (1- (1/ (1+exp(-100 * (S_at/theta_m -1)))))*(1-1/(1+exp(-100*((1000 * S_yt) /(theta_gx *S_gt)-1))))
dSgt_dt = dSbt_dt * theta_gb * (1 / (1 + exp(-100 * (S_at/theta_a-1)))) * (1 - (1/(1+exp(-1 * 100 * (S_at/theta_ts -1)))))
dSyt_dt = dSbt_dt * (1/ (1+exp(-100* (S_at/theta_a -1))))
D_t = 4098 * 0.6108 * exp((17.27 * W_at)/(W_at+237.3))/(W_at + 237.3)**2
e_st = 0.6108 * (exp(17.27 * W_xt/(W_xt + 237.3)) + exp(17.27 * W_nt/(W_nt +237.3))) /2 
e_at = e_st * W_ht * 0.01
gamma_t = 0.665 * 10**(-3) * W_pt2
e0_t = (0.408 * D_t * W_st + gamma_t * 900* W_wt* (e_st - e_at)/(W_at + 273))/(D_t + gamma_t * (1+ 0.34 * W_wt))  
k_ct = k_i + (1-exp(-1 * theta_k * S_lt)) * (k_m - k_i + (k_i-k_e)/(exp(-100 * (S_at/theta_alpha - 1) ) +1) ) - (k_i - k_e) /(exp(-100 * (S_at/theta_alpha -1)) +1)
e_ct = e0_t * k_ct
dSwt_dt = W_rt/(exp(-100 * (S_wt/S_xl + 1) ) + 1) - e_ct * f_wt
F = Matrix([dSvt_dt,dSat_dt,dSlt_dt,dSbt_dt,dSgt_dt,dSyt_dt,dSwt_dt ])

sample_arr = []
for v, a, l, b,g,y,w, t in zip(sol[:, 0], sol[:, 0], sol[:, 0], sol[:, 0],sol[:, 0], sol[:, 0],sol[:, 0], T ):
  a_n = F.jacobian([S_xl, theta_vr, theta_vd, theta_vo, 
              theta_vs, theta_vq, theta_ps, theta_a,
              theta_pr, theta_pm, theta_tr, theta_td, 
              theta_t0, theta_alpha, theta_el, theta_k,
              theta_sr, theta_sm, theta_rue, theta_m,
              theta_gx, theta_gb, theta_ts]).subs([(S_xl,150), (theta_vr,-0.28), (theta_vd,5), (theta_vo,4), 
              (theta_vs,1), (theta_vq,42), (theta_ps,0.8), (theta_a,36),
              (theta_pr,0.44), (theta_pm,10.67), (theta_tr,0.14), (theta_td,-7.6), 
              (theta_t0,24), (theta_alpha,0.30), (theta_el,18), (theta_k,0.75),
              (theta_sr,10), (theta_sm,35), (theta_rue,1.15), (theta_m,60),
              (theta_gx,45), (theta_gb,20), (theta_ts,15), (S_vt, v), (S_at, a), (S_lt,l ), (S_bt, b), (S_gt, g), (S_yt, y), (S_wt, w), 
              (W_at, Wat(t)), (W_pt1, Wpt1(t)), (W_pt2, Wpt2(t)), (W_st, Wst(t)), (W_xt, Wxt(t)),
              (W_nt, Wnt(t)),(W_ht, Wht(t)), (W_wt, Wwt(t)), (W_rt, Wrt(t)), 
              (k_m,0.5 ), (k_e, 0.5), (k_i, 0.5)])
  sample_arr.append(a_n)
  break


In [ ]:
# determinant = []
# sample_arr = np.array(sample_arr)
# for arr in sample_arr:
#   tempdash = np.array(arr, dtype=float)
#   temparr = np.matmul(tempdash, tempdash.T)
#   tempdet = np.linalg.det(temparr)
#   determinant.append(tempdet)

In [ ]:
from sympy.tensor.array import derive_by_array
J = derive_by_array(F, (S_xl, theta_vr, theta_vd, theta_vo, 
              theta_vs, theta_vq, theta_ps, theta_a,
              theta_pr, theta_pm, theta_tr, theta_td, 
              theta_t0, theta_alpha, theta_el, theta_k,
              theta_sr, theta_sm, theta_rue, theta_m,
              theta_gx, theta_gb, theta_ts))              

In [ ]:
derive_by_array(derive_by_array(F, (S_xl, theta_vr, theta_vd, theta_vo, 
              theta_vs, theta_vq, theta_ps, theta_a,
              theta_pr, theta_pm, theta_tr, theta_td, 
              theta_t0, theta_alpha, theta_el, theta_k,
              theta_sr, theta_sm, theta_rue, theta_m,
              theta_gx, theta_gb, theta_ts)), (S_xl, theta_vr, theta_vd, theta_vo, 
              theta_vs, theta_vq, theta_ps, theta_a,
              theta_pr, theta_pm, theta_tr, theta_td, 
              theta_t0, theta_alpha, theta_el, theta_k,
              theta_sr, theta_sm, theta_rue, theta_m,
              theta_gx, theta_gb, theta_ts))

In [1]:
from sympy import *
V, I = symbols("V I", cls=Function)
RC, t, C, Vs, L, R1, V0, I0 = symbols("RC t C Vs L R1 V0 I0")
system = [Eq(V(t).diff(t), -1/RC*V(t) + I(t)/C), Eq(I(t).diff(t), -R1/L*I(t) - 1/L*V(t) + Vs/L)]
ics = {V(0): V0, I(0): I0}
dsolve(system, [V(t), I(t)], ics=ics)

[Eq(V(t), -4*L*RC**3*Vs*Piecewise((2*C*L*RC*exp(t*(C*L**2*RC + C*L*R1*RC**2 - L*RC*sqrt(C**2*L**2 - 2*C**2*L*R1*RC + C**2*R1**2*RC**2 - 4*C*L*RC**2))/(2*C*L**2*RC**2))/(C*L + C*R1*RC - sqrt(C*(C*L**2 - 2*C*L*R1*RC + C*R1**2*RC**2 - 4*L*RC**2))), Ne(C*L + C*R1*RC - sqrt(C*(C*L**2 - 2*C*L*R1*RC + C*R1**2*RC**2 - 4*L*RC**2)), 0)), (t, True))*exp(-t*(C*L**2*RC + C*L*R1*RC**2 - L*RC*sqrt(C**2*L**2 - 2*C**2*L*R1*RC + C**2*R1**2*RC**2 - 4*C*L*RC**2))/(2*C*L**2*RC**2))/((C*(L - R1*RC) + sqrt(C*(C*(L**2 - 2*L*R1*RC + R1**2*RC**2) - 4*L*RC**2)))*(C*(L**2 - 2*L*R1*RC + R1**2*RC**2) - L*(4*RC**2 + sqrt(C*(C*(L**2 - 2*L*R1*RC + R1**2*RC**2) - 4*L*RC**2))) + R1*RC*sqrt(C*(C*(L**2 - 2*L*R1*RC + R1**2*RC**2) - 4*L*RC**2)))) + 2*RC*Vs*(C*(L**2 - 2*L*R1*RC + R1**2*RC**2) - L*(2*RC**2 + sqrt(C*(C*(L**2 - 2*L*R1*RC + R1**2*RC**2) - 4*L*RC**2))) + R1*RC*sqrt(C*(C*(L**2 - 2*L*R1*RC + R1**2*RC**2) - 4*L*RC**2)))*Piecewise((2*C*L*RC*exp(t*(C*L**2*RC + C*L*R1*RC**2 + L*RC*sqrt(C**2*L**2 - 2*C**2*L*R1*RC + C**2

In [4]:
from sympy import symbols, diff, sin
x, y, z = symbols('x y z', real=True)
f = 4*x*y + x*sin(z) + x**3 + z**8*y
diff(f, x)

3*x**2 + 4*y + sin(z)

In [5]:
f.diff(x)

3*x**2 + 4*y + sin(z)